In [1]:
from scripts.reward_learning import reward_learning
from scripts.reward_evaluation import get_accuracy
from scripts.ppo_model_training import get_dataset_iterator, prepare_model_for_PPO, prepare_reward_model_for_PPO, prepare_PPO_config, prepare_PPO_trainer, train_model
from scripts.model_evaluation import load_true_reward_model, load_model_and_tokenizer,  generate_response_from_model, evaluate_responses

/home/t-dasnirjhar/anaconda3/envs/trl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Anthropic + Gemma

### Anthropic + Gemma : Small Epochs

In [2]:
model_name = 'Gemma'
dataset_name = 'Anthropic'
output_folder = './output'
small = True
info = 'le' # identifier for large epoch

In [3]:
samples_per_epoch = 100
epochs = 20
id_apo = 'apo'

In [4]:
reward_learning(dataset_name, model_name, samples_per_epoch, epochs, output_folder, id_apo, small, info)

print('-'*50)
get_accuracy(model_name, dataset_name, output_folder, small, id_apo, info)


100%|██████████| 20/20 [00:02<00:00,  7.62it/s]


--------------------------------------------------
apo accuracy: 94.55%


In [4]:
samples_per_epoch = 100
epochs = 20
id_rand = 'random'

In [6]:
reward_learning(dataset_name, model_name, samples_per_epoch, epochs, output_folder, id_rand, small, info)

print('-'*50)
get_accuracy(model_name, dataset_name, output_folder, small, id_rand, info)

100%|██████████| 20/20 [00:01<00:00, 19.44it/s]


--------------------------------------------------
random accuracy: 83.0%


In [ ]:
ds = get_dataset_iterator(dataset_name, model_name)
model_apo, tokenizer = prepare_model_for_PPO(model_name)
reward_model_apo, rm_tokenizer_apo = prepare_reward_model_for_PPO(model_name, dataset_name, output_folder, id_apo, info)
ppo_config = prepare_PPO_config(model_name)
ppo_trainer_apo, gen_kwargs = prepare_PPO_trainer(ppo_config, model_apo, tokenizer, ds)
train_model(ppo_trainer_apo, gen_kwargs, tokenizer, reward_model_apo, rm_tokenizer_apo, output_folder, model_name, dataset_name, id_apo, max_iter=32, small=small, info=info)

# Free up GPU
del model_apo
del tokenizer
del reward_model_apo
del rm_tokenizer_apo
del ppo_config
del ppo_trainer_apo
del gen_kwargs

In [ ]:
ds = get_dataset_iterator(dataset_name, model_name)
model_rand, tokenizer = prepare_model_for_PPO(model_name)
reward_model_rand, rm_tokenizer_rand = prepare_reward_model_for_PPO(model_name, dataset_name, output_folder, id_rand, info)
ppo_config = prepare_PPO_config(model_name)
ppo_trainer_rand, gen_kwargs = prepare_PPO_trainer(ppo_config, model_rand, tokenizer, ds)
train_model(ppo_trainer_rand, gen_kwargs, tokenizer, reward_model_rand, rm_tokenizer_rand, output_folder, model_name, dataset_name, id_rand, max_iter=32, small=small, info=info)

# Free up GPU
del model_rand
del tokenizer
del reward_model_rand
del rm_tokenizer_rand
del ppo_config
del ppo_trainer_rand
del gen_kwargs

In [ ]:
model, tokenizer = load_model_and_tokenizer(model_name, dataset_name, output_path=output_folder, id='apo', small=small, info=info)

generate_response_from_model(model, tokenizer, dataset_name, model_name, 'apo', output_folder, small, 2000, info)

del model
del tokenizer

In [ ]:

reward_processor = load_true_reward_model(model_name, dataset_name)
evaluate_responses(reward_processor, dataset_name, model_name, 'apo', output_folder, small, info)
del reward_processor

In [ ]:
model, tokenizer = load_model_and_tokenizer(model_name, dataset_name, output_path=output_folder, id='random', small=small, info=info)

generate_response_from_model(model, tokenizer, dataset_name, model_name, 'random', output_folder, small, 2000, info)

del model
del tokenizer

In [ ]:

reward_processor = load_true_reward_model(model_name, dataset_name)
evaluate_responses(reward_processor, dataset_name, model_name, 'random', output_folder, small, info)
del reward_processor